In [ ]:
import os

import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split

import lifelines

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.lines as mlines

# Autoencoding colorectal cancer

First, load the mRNA data

In [ ]:
DATADIR = "data"

In [ ]:
mrna = pd.read_csv(os.path.join(DATADIR, "crc-mrna.csv.gz"), index_col=0)
mrna.head()

Observe the skewed distributions of the features (genes). These will need to be normalized in some way to get them to work with neural networks.

In [ ]:
fig,ax = plt.subplots()
for gene in np.random.choice(mrna.index, 10, replace=False):
    sns.distplot(mrna.loc[gene], ax=ax, label=gene)
plt.legend()
ax.set_xlabel("RPKM")
ax.set_xlim(-1,1000)

Normally, we'll log-transform RPKM values to get them to look more normal

In [ ]:
mrna_log = np.log2(mrna+1)

fig,ax = plt.subplots()
for gene in np.random.choice(mrna.index, 10, replace=False):
    sns.distplot(mrna_log.loc[gene], ax=ax, label=gene)
plt.legend()
ax.set_xlabel("log(RPKM+1)")
# ax.set_xlim(-1,1000)

Next, we'll scale these so they all are closer to 0

In [ ]:
mrna_scaled = pd.DataFrame(
    scale(mrna_log.copy(), axis=1),
    index=mrna.index,
    columns=mrna.columns
)

fig,ax = plt.subplots()
for gene in np.random.choice(mrna.index, 10, replace=False):
    sns.distplot(mrna_scaled.loc[gene], ax=ax, label=gene)
plt.legend
ax.set_xlabel("scaled RPKM")

Next, load the subtype data

In [ ]:
subtypes = pd.read_csv(os.path.join(DATADIR, "crc-subtypes.csv.gz"), index_col=0)
subtypes.head()

Just to get a rough idea of what this data looks like, let's make a scatter plot

In [ ]:
# Define a dictionary mapping sample IDs to their subtype, from the table above
sample2subtype = dict(zip(subtypes.index, subtypes.cms_label))

# Define a dictionary mapping subtypes to colors
subtype2color = dict(zip(sorted(subtypes.cms_label.unique()), sns.color_palette()))

# Define an array of colors for all samples, using the two dictionaries defined just above
sample_colors = [subtype2color.get(sample2subtype.get(s, "NOLBL")) for s in mrna.columns]

In [ ]:
fig,ax = plt.subplots(figsize=(8,8))

# 2D PCA and scatter plot
pd.DataFrame(
    PCA(2).fit_transform(mrna_log.T),
    index=mrna_scaled.columns,
    columns=["PC1", "PC2"]
).plot.scatter("PC1", "PC2", color=sample_colors, s=40, ax=ax)

# Add figure legend
cms_legend_handles = [mlines.Line2D([],[], label=label, color=color, markersize=10, marker="o", linewidth=0) for label, color in subtype2color.items()]
ax.legend(handles=cms_legend_handles)

For another quick look at the data, let's explore these subtypes' survival rates. First, load the survival data.

In [ ]:
survival = pd.read_csv(os.path.join(DATADIR, "crc-survival.csv.gz"), index_col=0)
survival.head()

In [ ]:
fig,ax = plt.subplots(figsize=(10,6))

# Instantiate a Kaplam Meier Fitter object
kmf = lifelines.KaplanMeierFitter()

# For each CMS subtype, plot the KM estimate
for subtype in [f"CMS{i}" for i in range(1,5)]:
    # Select the patients with this subtype
    patients = subtypes[subtypes.cms_label==subtype].index
    
    # Select the patients with this subtype who also have survival data in the table
    patients_with_survival = set(patients) & set(survival.index)
    s = survival.loc[patients_with_survival]
    
    # Fit and plot the KM estimate
    kmf.fit(s.duration, s.observed, label=subtype)
    kmf.plot(ax=ax, ci_show=False, show_censors=True, linewidth=5)

----------------

# Fitting an autoencoder

In [ ]:
import tensorflow as tf
from tensorflow import keras

# Set some sensible limits when running on CPU on a shared server
tf.keras.backend.clear_session()
tf.config.threading.set_intra_op_parallelism_threads(2)
tf.config.threading.set_inter_op_parallelism_threads(2)

# Print the tensorflow version
tf.version.VERSION

First, define a model:

In [ ]:
input_size = mrna.shape[0] # The dimension of the input layer, is the number of genes
embedding_size = 2 # The embedding size, of the reduced dimension (dimension of latent soace)

# Definint the autoencoder layers
mrna_input = keras.layers.Input(shape=(input_size,), name="input")
hidden = keras.layers.Dense(embedding_size, activation="sigmoid", name="hidden")(mrna_input)
output = keras.layers.Dense(input_size, activation="sigmoid", name="reconstruction")(hidden)

# Defining the end-to-end autoencoder model
ae = tf.keras.Model(mrna_input, output, name="Vanilla autoencoder")

# Compiling the model, using the Adam optimized, and MSE loss
ae.compile(optimizer=tf.optimizers.Adam(), loss=tf.losses.mean_squared_error)

In [ ]:
ae.summary()

Before training the model, let's split the data into training and testing!

In [ ]:
# Split the data into training and testing data, keeping 90% for training
x_train, x_test = train_test_split(mrna_scaled.T, train_size=.9)

In [ ]:
# Using Keras to train the autoencoder. Not that x==y, because we are teaching it to reconstruct!
ae.fit(x=x_train, y=x_train, validation_data=[x_test, x_test], epochs=500)

In [ ]:
# Plot the training loss
pd.DataFrame(ae.history.history).plot()

Now since we're satisfied with the overall performance of this autoencoder, let's split it up in encoder and decoder. After all, it's mainly the encoder we're interested in, in order to perform dimensionality reduction...

In [ ]:
# The encoder model is a keras model like the full autoencoder
# It reuses the same layers, but only goes from input to latent space!
encoder = keras.Model(mrna_input, hidden, name="Encoder")
encoder.summary()

In [ ]:
# Define a dataframe holding autoencoded data
z = pd.DataFrame(
    encoder.predict(mrna_scaled.T), # using the encoder model's predict() method!
    index=mrna_scaled.columns,
    columns=[f"LF{i}" for i in range(1, embedding_size+1)]
)

# a 2D scatter plot
z.plot.scatter("LF1", "LF2", color=sample_colors)

Not super impressive. How about using a larger network?

In [ ]:
input_size = mrna.shape[0]
embedding_size = 100

# Define a function that returns autoencoders and encoders
def make_vanilla_autoencoder(input_size, embedding_size):
    # Define the layers
    mrna_input = keras.layers.Input(shape=(input_size,), name="input")
    hidden = keras.layers.Dense(embedding_size, activation="sigmoid", name="hidden")(mrna_input)
    output = keras.layers.Dense(input_size, activation="sigmoid", name="reconstruction")(hidden)
    
    # Define the end-to-end autoencoder model, and compile it so it can be trained
    ae = tf.keras.Model(mrna_input, output, name="Vanilla autoencoder")
    ae.compile(optimizer=tf.optimizers.Adam(), loss=tf.losses.mean_squared_error)
    
    # Define the input-hidden Encoder network
    encoder = keras.Model(mrna_input, hidden, name="Encoder")
    
    # Return both the full autoencoder model, and the encoder network
    return ae, encoder

ae, encoder = make_vanilla_autoencoder(input_size, embedding_size)

In [ ]:
ae.summary()

In [ ]:
ae.fit(x=x_train, y=x_train, validation_data=[x_test, x_test], epochs=1000)

In [ ]:
pd.DataFrame(ae.history.history).plot()

Here, it looks like the validation loss is actually increasing, though the training loss is decreasing, after approximately epoch 400:

In [ ]:
pd.DataFrame(ae.history.history).plot()
plt.xlim(0,700)
plt.ylim(.575,.7)

A good way to deal with this is by early stopping. In tensorflow 2.0 / keras, we have this easy-to-use callback:

In [ ]:
# Define new autoencoder and encoder models using the function we wrote above
ae, encoder = make_vanilla_autoencoder(input_size, embedding_size)

# Fit the model, but this time also use the callbacks argument
ae.fit(
    x=x_train,
    y=x_train,
    validation_data=[x_test, x_test],
    epochs=1000,
    callbacks=[
        keras.callbacks.EarlyStopping( # EarlyStopping is conveniently implemented as a standard keras callback
            monitor="val_loss", # We want early stopping to look at the validation loss
            patience=3 # and wait 3 epochs before truly stopping
        )
    ]
)

In [ ]:
pd.DataFrame(ae.history.history).plot()

Here, the model stopped training after the validation loss leveled off. Now, let's use it, by using the encoder network:

In [ ]:
encoder.summary()

In [ ]:
# Define a dataframe with the latent factor representation of the input
z = pd.DataFrame(
    encoder.predict(mrna_scaled.T), # Using the encoder's predict() function!
    index=mrna_scaled.columns,
    columns=[f"LF{i}" for i in range(1, embedding_size+1)]
)
z.head()

Now, as `z` has 100 latent factors in it, let's plot a PCA of those, in order to get a 2D visualization:

In [ ]:
fig,ax = plt.subplots(figsize=(8,8))

# 2D PCA of the latent space and a scatter plot
pd.DataFrame(
    PCA(2).fit_transform(z),
    columns=["PC1", "PC2"],
    index=z.index
).plot.scatter(
    "PC1",
    "PC2",
    color=sample_colors,
    s=40,
    ax=ax
)

ax.legend(handles=cms_legend_handles)

So, using a simple autoencoder, we've reduced the dimensionality from 1,000 genes to 100 latent factors. Then, we used PCA to get 2 dimensions, which we plot here. Whether or not these are better than PCA on the original data is left as an exercise.

However, we could also use this reduced dimension latent space to define our own clusters, i.e. CRC subtypes. For instance, using k-means:

In [ ]:
# Use k-means (k=4) to define clusters of the data
ae_clusters = pd.Series(KMeans(4).fit_predict(z), index=z.index)

In [ ]:
# How many samples are in each cluster?
ae_clusters.value_counts().plot.bar()

In [ ]:
# Define a dictionary mapping each sample to its assigned cluster
sample2cluster = dict(zip(ae_clusters.index, ae_clusters))

# Defien a dictionary mapping clusters to colors
cluster2color = dict(zip(sorted(ae_clusters.unique()), sns.color_palette()))

# Define an array with each sample's cluster color, according to the dicts above
sample_colors_ae_clusters = [cluster2color.get(sample2cluster.get(s)) for s in ae_clusters.index]

In [ ]:
fig,ax = plt.subplots(figsize=(8,8))

# 2D PCA and scatter plot
pd.DataFrame(
    PCA(2).fit_transform(z),
    columns=["PC1", "PC2"],
    index=z.index
).plot.scatter(
    "PC1",
    "PC2",
    color=sample_colors_ae_clusters, # colored by the cluster colors
    s=40,
    ax=ax
)

# Adding a figure legend
ae_clusters_legend_handles = [mlines.Line2D([],[], label=label, color=color, markersize=10, marker="o", linewidth=0) for label, color in cluster2color.items()]
ax.legend(handles=ae_clusters_legend_handles)

So using k-means, we've split the patients into four groupings. Do these groupings reveal anything interesting? Let's for instance examine their survival statistics:

In [ ]:
fig,ax = plt.subplots(figsize=(10,6))

kmf = lifelines.KaplanMeierFitter()

# For each cluster
for cl in ae_clusters.unique():
    # The patienrs in this cluster
    patients = ae_clusters[ae_clusters==cl].index
    
    # The patients in this cluster who also have survival data
    patients_with_survival = set(patients) & set(survival.index)
    s = survival.loc[patients_with_survival]
    
    # Estimate and plot kaplan meier curve
    kmf.fit(s.duration, s.observed, label=cl)
    kmf.plot(ax=ax, ci_show=False, show_censors=True, linewidth=5)

---------------------

# Exercises

## Regularization

### Exercise 1 (EASY): add l1 regularization to the autoencoder

A vanilla autoencoder like this is probably going to produce a fairly dense network, i.e. one where most nodes are connected with most nodes. We can examine the weights of the network like this:

In [ ]:
sns.distplot(ae.layers[1].get_weights()[0].flatten(), kde=False)
plt.xlabel(r"$\|w\|$")

This distribution of weights is approximately normal.

Hint: `keras.layers.Dense?`, or https://keras.io/regularizers/

In [ ]:
# your code here

### Exercise 2 (EASY): De-noising autoencoder (DAE)

Recall that a denoising autoencoder adds noise to the input **but not to the output** so that it learns to reconstruct a noise-free sample from noisy input. The DAE learns not only to reconstruct noise free samples, but because all noise applied to an input should result in the same output, it also results in the same latent space representation. Hence, the DAE learns that samples which are _within noise_ of each other, should have the same latent representations. This feature makes DAEs learn smooth manifolds, a great advantage over vanilla AEs.

A naive way to implement a DAE is simply to add noise to `x` for the input, and train it using `model.fit(x+noise, x)`. This approach, however, misses out on the manifold-building advantages of DAEs, as each sample only gets one characteristic "noise" attached to it, no matter how many epochs we train it for.

**Implement a DAE which does not have this drawback**.

Hint: `keras.layers.GaussianNoise?`

In [ ]:
# your code here

### Exercise 3 (CHALLENGING): Variational Autoencoder (VAE)

Recall that a variational autoencoder learns **distributions** for latent factors, rather than a deterministic mapping, i.e. $z \sim q(z|x,w)$. Setting it as a normally distributed variable, this means learning a **mean** and **standard deviation** for each latent factor, rather than a simple value. Using the reparametrization trick, this is achieved by having two parallel layers representing the mean and std, followed by a sampling procedure to get `z`s.

Finally, the KL-divergence of $q(z|x,w)$ from the prior $p(z)$ (a unit gaussian) needs to be added to the loss function.

**Implement a VAE**.

_Tip:_ Typically when implementing VAEs, a network is taught to estimate $\mu$ and $log(\sigma)$. This results in more stable estimates.

_Hint:_ the KL divergence of a normally distributed $q(z|x,w)$ from a unit normal is given by:

$$
KL(\mathcal{N}(\mu, \Sigma)||\mathcal{N}(0,1)) = \frac{1}{2} \sum_k \left( \sigma_k + \mu_k^2 - log(\sigma_k) - 1 \right)
$$

This term will need to be added to the model's **loss**, in addition to the reconstruction loss, to complete the ELBO. Hint: `model.add_loss?`.

_Another hint:_ Sampling from a distribution (using the reparametrization trick) can be achieved using a custom function layer. This layer can take the $\mu$ and $\sigma$ as inputs, and sample from a distribution given by these parameters. Hint: `keras.layers.Lambda?` and `tf.random.normal?`.

In [ ]:
# your code here

### Exercise 3A (MODERATE): VAE with deterministic encoder

VAE encoders (the encoder network of a vae) don't produce encodings directly --- they produce, for each latent factor, the variance and mean of a normal distribution. It is by sampling from that distribution that latent factors are obtained. However, this means that every time the encoder is called, on the same data, it will produce a different sample of the latent space representation! While it is important to keep this behavior during training, keeping it in the _use_ phase (when we get latent space representations of data) can present a problem for reproducability.

**How would you address this? What kind of a deterministic encoder model could be defined that makes use of the trained VAE?**

In [ ]:
# your code here

### Exercise 4 (EXTRA CHALLENGING): Contractive autoencoder (CAE)

The CAE aims to give close latent representations to points which are close in input space. This is achieved by penalizing the frobenius norm of the jacobian of hidden neuron activations with respect to inputs, i.e. the gradient of the hidden layer with respect to the input. For an intuition behind this, we don't want small perturbations in the input to result in large perturbations in the latent space. The gradient of the hidden with respect to the input represents the size of latent space perturbations resulting from input perturbations. With a tuning parameter $\lambda$, this results in a loss function $\mathcal{L} = MSE + \lambda\|J_x(z)\|_F$.

**Implement a contractive autoencoder**.

_Hint:_ Take a look at the TensorFlow subclassing API. Specifically, how to add losses: https://www.tensorflow.org/guide/keras/custom_layers_and_models#layers_recursively_collect_losses_created_during_the_forward_pass.

In [ ]:
# your code here

-------------

## Deep autoencoders

### Exercise 5 (EASY): Deep Vanilla Autoencoder (Stacked AEs)

**Implement an n-layer vanilla autoencoder**.

**Question: Is there a disadvantage to stacking AEs?**

In [ ]:
# your code here

### Exercise 6 (CHALLENGING): Stacked Variational Autoencoders

**Implement stacked VAEs**.

**Variant 1: several "regular" layers with one "variational" layer in the middle**

**Variant 2: variational at each layer**

In [ ]:
# your code here

--------------

## Model Interpretation

### Exercise 7 (EASY): Explain which genes are associated with each latent factor

Using the weights from input genes to hidden neurons, we can explain which genes are behind each latent factor.

**Question 1:** are l1-regularized autoencoders sparser than "vanilla" autoencoders? Can a relationship be established between the regularization parameter and the number of genes associated with each latent factor?

**Question 2:** Are DAEs, VAEs, or CAEs better at creating sparser autoencoders?

In [ ]:
# your code bere

### Exercise 8 (SLIGHTLY CHALLENGING): Explaining deep autoencoders

**Question**: How would you establish the connection between input genes and latent factors in a deep autoencoder?

In [ ]:
# your code here

### Exercise 9 (SLIGHTLY CHALLENGING): pruning over-parameterized autoencoders

We've seen the advantages of over-parameterizing autoencoders, e.g. by specifying a larger hidden dimension than we think is really latent in the data. The challenge remains to decide how to prune the "noise" latent factors in our model.

1. Implement model pruning by _proportion of variance explained_, i.e. only keep those latent factors which explain a significant amount of variance in the input data.
    1. (extra points) can this be done while training?
2. Implement model pruning by gene set enrichment, i.e. associate each latent factor with genes, and only keep the ones which are associated with interesting gene sets
3. Implement model pruning by clinical relevance, i.e. only keep latent factors which are predictive of patient survival
4. Can you think of any other ways to subspace the large latent space we've inferred?

In [ ]:
# your code here

## Combining models

### Exercise 10 (CHALLENGING): Using a pre-trained autoencoder as a component in another neural network

A major advantage of autoencoders being unsupervised learning techniques is, they do not require any labels, and can be used to learn patterns in data which is unlabeled. However, sometimes we want to train models to perform supervised learning tasks, e.g. classifying patients into a CMS subtype based on their molecular profiles.

We have molecular profiles for more patients than we have subtype labels for:

In [ ]:
mrna_patients = set(mrna.columns)
subt_patients = set(subtypes[subtypes.cms_label.isin({f"CMS{i}" for i in range(1,5)})].index)
print(f"Out of {len(mrna_patients)} patients with mRNA seq, we have CMS labels for {len(mrna_patients & subt_patients)}.")

It would be a shame to throw away the molecular profile of the 100 patients (representing 20% of the data at hand!), if we wanted to train a classifier to predict the CMS subtype from the mRNA profile.

One way to make use of the data, is to **pre-train** an autoencoder on all of the data (both labeled and unlabeled). The resulting encoder network will have learned to create a meaningful representation of the input, and it will have made use of the unlabeled samples too in doing so. Then in the next step, we can use this pre-trained encoder network, put another dense/softmax layer at the end, and teach that thing to classify tumors into their CMS subtypes based on their mRNA profiles.

In [ ]:
# your code here

### Exercise 11 (MODERATE): Multi-omics autoencoder

We've also made available mutation and copy number data for the colorectal tumors:

In [ ]:
muts = pd.read_csv(os.path.join(DATADIR, "crc-muts.csv.gz"), index_col=0)
cnvs = pd.read_csv(os.path.join(DATADIR, "crc-cnvs.csv.gz"), index_col=0)

**Implement a multi-omics integration autoencoder**

In [ ]:
# your code here